# TODO:
### 24Oct23 - 31Oct23
- use lat-lon for distance and make that the x axis of the layer
    - distance from... what exactly?
        - the exact placement of $d_0$ may not actually matter as long as it is noted on the plot that it is arbitrary (or however it is actually placed)
            - i.e. it is only useful for showing scale
    - this would put the plot in meters on both axes if reading the right side y axis scale
- get the distance between crossover points
    - *GPS time is not a useful measure of ~distance~ anything*
    - use the lat-lon coordinates to get the distance between points
        - i.e. get the 4 endpoints of the 2 segments and use the distance between them
    - **distance between points in the 2018 Antarctica DC8 20181030_01 flight is about 14.9m**
        - need to make distance step a variable that is read from this because it may change flight to flight
- find the slope in both directions across the segments
    - probably want to go out a bit from the crossover point to get a better slope
        - e.g. 2 or 3x crossover point distance
    - may have to normalize the slope if jagged
- find the plane heading on each segment
    - check for point to point consistency - wandering gps
        - i.e. make sure that the points in the segment are actually in a straight line with the surrounding points
        - can probably implement an if statement for this
            - `if heading(segment) ~= heading(segment + a_few_points_either_end), then segment is straight`
            - `else, use heading(segment + a_few_points_either_end)`


In [1]:
import math
from layer_class import Twtt_Posit
from library import *

In [2]:
zoom = False 
# zoom = True
seg_length = 100
season = "2018_Antarctica_DC8"
# season = "2016_Antarctica_DC8"
flight = "20181030_01"  # the flight date and frame number
# flight = "20181103_01"
# flight = "20181112_02"
# flight = "20161024_05"
file_name = "layer_export_" + flight + ".pickle"

# TODO: make map plotter center on crossover point n. default to centering on the first crossover point if no n is given
# TODO: choose a crossover point or save the data for them or something. Do we already do this? Probably. Very tired.
    # Who is this we?

In [3]:
layers = read_layers(file_name)  # read in the layers from the pickle file

intersection_points, intersection_indices, segment_ends = cross_point(layers[0], seg_length, quiet=False)
# find the crossover points

posit = Twtt_Posit(layers[1], season, flight, intersection_indices)
# create a Twtt_Posit object to store the crossover point data
# print(f"posit.layer_name: {posit.layer_name}")

save_posit(posit)

Reading pickle file...
--------------------
Surface
Bottom
Test
--------------------

Finding crossover point...
--------------------
Dividing the path into segments of length 100...
Number of segments: 547
Checking for intersections...
Segments 54 and 122 intersect near (-87.97980253666243, -130.00293379604554)

Checking for a more precise intersection...
segment 54 start: 5400, segment 1 end: 5500
segment 122 start: 12200, segment 2 end: 12300
Segments 54 and 122 intersect near indices 5481 and 12389
This corresponds roughly to the lat-lon: (-87.97979626378307, -130.00290123313295)
Number of intersections: 1
Number of rough intersections: 1
Number of intersection indices: 1
Indices: [[5481, 12389]]
Index: [5481, 12389]
--------------------

Saving posit...
--------------------
posit.pickle saved in local directory of this python file.
--------------------


In [4]:
def gps_time_to_seconds(gps_time):
    """
    :param gps_time: the gps time in seconds
    :return: the gps time in seconds
    """
    return gps_time * 10 ** -9

In [7]:
print(layers[0].gps_time[0])
# time is epoch seconds from 1970

1540923785.6594396


In [4]:
# print(f"Segment ends: \n{segment_ends}\n")
# latlon_dist(segment_ends[0][0][0], segment_ends[0][0][1])
# latlon_dist(segment_ends[0][1][0], segment_ends[0][1][1])
# print(f"Distance between segment 1 endpoints: {round(latlon_dist(segment_ends[0][0][0], segment_ends[0][0][1]),2)} m")
# print(f"Distance between segment 2 endpoints: {round(latlon_dist(segment_ends[0][1][0], segment_ends[0][1][1]),2)} m")

# print(layers[0].gps_time[intersection_indices[0][0]])

# find the plane's velocity in m/s using the distance between the endpoints and the time between the endpoints
time1 = gps_time_to_seconds(layers[0].gps_time[intersection_indices[0][0]])
time2 = gps_time_to_seconds(layers[0].gps_time[intersection_indices[0][1]])
print(f"time1: {time1}, time2: {time2}")
d_time = time2 - time1
print(f"Time between segment 1 endpoints: {d_time} s")
dist = latlon_dist(segment_ends[0][0][0], segment_ends[0][0][1])
print(f"Distance between segment 1 endpoints: {round(dist,2)} m")
vel = dist / d_time
print(f"Plane velocity: {round(vel,2)} m/s")


# vel_ms, vel_kh = plane_velocity(segment_ends[0][0][0], segment_ends[0][0][1], time1,time2)


# print(f"Plane velocity: {round(vel_ms,2)} m/s or {round(vel_kh,2)} km/h")


# print(layers[1].layer_name)
slope = slope_around_index(layers[1], intersection_indices[0][0], 2)
print(f"{round(slope,2)}")

NameError: name 'gps_time_to_seconds' is not defined

In [40]:
"""
Clean this up and put it in a function in the library file (?)
"""

print("Comparing my depth to CReSIS depth...")
print("--------------------")
twtt_at_intersect = twtt_at_point(layers[1], layers[0], intersection_indices, quiet=True)
# find the twtt at the crossover points
twtt_difference_at_intersect = twtt_at_intersect[0][0] - twtt_at_intersect[0][1]
# find the difference in twtt at the crossover points
print(f"twtt difference at crossover point: {twtt_difference_at_intersect} ns")

my_refractive_index = 1.77
# my_depth_1 = twtt_to_depth(twtt_at_intersect[0][0], my_refractive_index)
# my_depth_2 = twtt_to_depth(twtt_at_intersect[0][1], my_refractive_index)
#
# print(f"By my refractive index: {my_refractive_index}")
# print(f"depth at crossover point on segment 1: {my_depth_1} m")
# print(f"depth at crossover point on segment 2: {my_depth_2} m")
# print(f"difference: {my_depth_1 - my_depth_2} m")

# loop through all intersections and print the depth at each
for i in range(len(intersection_indices)):
    my_depth_1 = twtt_to_depth(twtt_at_intersect[i][0], my_refractive_index)
    my_depth_2 = twtt_to_depth(twtt_at_intersect[i][1], my_refractive_index)
    print(f"depth at crossover point {i} on segment 1: {my_depth_1} m")
    print(f"depth at crossover point {i} on segment 2: {my_depth_2} m")
    print(f"difference: {my_depth_1 - my_depth_2} m")

print(f"crossover point lat-long: {intersection_points[0]}")

Comparing my depth to CReSIS depth...
--------------------
twtt difference at crossover point: -1.5131660511637695e-07 ns
depth at crossover point 0 on segment 1: 2701.0368867503407 m
depth at crossover point 0 on segment 2: 2713.8514565198475 m
difference: -12.814569769506761 m
depth at crossover point 1 on segment 1: 2923.8402742772882 m
depth at crossover point 1 on segment 2: 2931.453882025299 m
difference: -7.613607748010963 m
depth at crossover point 2 on segment 1: nan m
depth at crossover point 2 on segment 2: nan m
difference: nan m
depth at crossover point 3 on segment 1: 2826.3315070057624 m
depth at crossover point 3 on segment 2: 2781.953713377351 m
difference: 44.37779362841138 m
depth at crossover point 4 on segment 1: 1559.4255870469394 m
depth at crossover point 4 on segment 2: 1573.074077207646 m
difference: -13.648490160706615 m
depth at crossover point 5 on segment 1: 282.21367970766346 m
depth at crossover point 5 on segment 2: 271.0696894612235 m
difference: 11.14

In [41]:
cresis_refractive_index = math.sqrt(3.15)
# cresis_refractive_index = 1.785
cresis_depth_1 = twtt_to_depth(twtt_at_intersect[0][0], cresis_refractive_index)
cresis_depth_2 = twtt_to_depth(twtt_at_intersect[0][1], cresis_refractive_index)

print(f"By CReSIS refractive index: {cresis_refractive_index}")
print(f"depth at crossover point on segment 1: {cresis_depth_1} m")
print(f"depth at crossover point on segment 2: {cresis_depth_2} m")
print(section_break)
"""
End of the cleanup (for now)
"""

By CReSIS refractive index: 1.7748239349298849
depth at crossover point on segment 1: 2693.6955240785455 m
depth at crossover point on segment 2: 2706.475264111138 m
--------------------


'\nEnd of the cleanup (for now)\n'

In [42]:
def plot_layers_at_cross(layers, intersection_indices, intersection_points, zoom=False, refractive_index=1.77, cross_index=0):
    """
    :param layers: a list of Layer objects
    :param intersection_indices: a list of indices in the lat-lon arrays where the flight path
    crosses over itself
    :param intersection_points: a list of lat-lon points where the flight path crosses over itself
    :return: nothing (plots the layers and the map)
    """
    plt.figure(figsize=(24, 12), layout='tight')
    print("Plotting layers and map...")
    print("--------------------")
    print("Adjusting for surface twtt...")
    for layer in layers:
        corrected_layer = layer.twtt - layers[0].twtt
        layer.twtt_corrected = corrected_layer

    # ax2 will be the layer plot
    plt.subplot(1, 2, 1)

    # plot the layer depths vs index for 500 points before and after the first
    # crossover point for each layer.
    # also plot the layer depths vs index for 500 points before and after the
    # second crossover point for each layer.
    offset = 500
    # plot the corrected twtt for each layer
    plt.plot(layers[0].twtt_corrected[intersection_indices[0][0] - offset:intersection_indices[cross_index][0] + offset],
             label=layers[0].layer_name)
    plt.plot(layers[1].twtt_corrected[intersection_indices[0][0] - offset:intersection_indices[cross_index][0] + offset],
             label=layers[1].layer_name + ' segment 1')
    plt.plot(layers[1].twtt_corrected[intersection_indices[0][1] - offset:intersection_indices[cross_index][1] + offset],
             label=layers[1].layer_name + ' segment 2')

    # plot uncorrected twtt for each layer
    # plt.plot(layers[0].twtt[intersection_indices[0][0] - offset:intersection_indices[0][0] + offset],
    #             label=layers[0].layer_name)
    # plt.plot(layers[1].twtt[intersection_indices[0][0] - offset:intersection_indices[0][0] + offset],
    #             label=layers[1].layer_name + ' segment 1')
    # plt.plot(layers[1].twtt[intersection_indices[0][1] - offset:intersection_indices[0][1] + offset],
                # label=layers[1].layer_name + ' segment 2')


    # invert the y-axis because the twtt increases with depth
    plt.gca().invert_yaxis()
    # plot the crossover point on the plot
    plt.scatter(offset, twtt_at_point(layers[1], layers[0],
                                      intersection_indices, quiet=True)[0][0], color='red',
                label='X Point 1')
    plt.scatter(offset, twtt_at_point(layers[1], layers[0],
                                      intersection_indices, quiet=True)[0][1], color='green',
                label='X Point 2')
    # plot a line at the crossover point
    plt.axvline(x=offset, color='black', label='X Point', linestyle='--', linewidth=0.3)

    # set the y axis to be in nanoseconds instead of seconds
    plt.ylabel("Adjusted Two Way Travel Time (ns)")
    plt.xlabel("Index")

    # force the y values to be displayed in 1e-6 ticks (microseconds) instead of 1e-5 ticks (tens of microseconds)
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0, 0), useMathText=True)

    def s_to_ms(x, pos):
        """
        :param x: the x value
        :param pos: the position
        :return: the x value in milliseconds
        """
        return '%1.1f' % (x * 1e6)

    # set the y axis to be in microseconds instead of seconds
    plt.gca().yaxis.set_major_formatter(plt.FuncFormatter(s_to_ms))

    # make the right side y axis show the depth in meters by converting the twtt to depth using the refractive index
    min_y, max_y = plt.ylim()
    n = refractive_index
    c = 299792458  # m/s
    v = c / n
    # depth = twtt * v / 2
    scale_factor = v / 2
    print(f"scale factor: {scale_factor}")
    plt.twinx()
    plt.ylim(min_y * scale_factor, max_y * scale_factor)
    plt.ylabel("Depth (m)")
    
    # make the top of the x axis be the distance in meters by converting the lat-lon to distance using the haversine formula
    min_x, max_x = plt.xlim()
    scale_factor = latlon_dist((layers[0].lat[0], layers[0].lon[0]), (layers[0].lat[1], layers[0].lon[1]))
    print(f"scale factor: {scale_factor}")
    plt.twiny()
    plt.xlim(min_x * scale_factor, max_x * scale_factor)
    plt.xlabel("Distance (m)")
    

    plt.title("Adjusted Two Way Travel Time vs Index")
    plt.legend()

    """
    plot the map
    """
    plt.subplot(1, 2, 2)

    # TODO: make the right pane a zoomed in map centered around the X point
    #  with a small zoomed out map in the corner
    # TODO: adjust time scale to be in nanoseconds instead of seconds
    # zoom_out_to_continent = False


    # # this code sets up a polar stereographic map of antarctica with the South Pole in the center
    # zoom_out_to_continent = not zoom
    # if zoom_out_to_continent:
    #     bound_lat = -65
    # else:
    #     bound_lat = -87
    # # plot the lat-lon map for one of the layers in antarctica
    # # print("Plotting lat-lon map...")
    # # print("--------------------")
    # m = Basemap(projection='spstere', boundinglat=bound_lat, lon_0=180, resolution='l')
    # m.drawcoastlines()
    # m.fillcontinents(color='grey', lake_color='aqua')
    # m.drawparallels(np.arange(-80., 81., 20.))
    # m.drawmeridians(np.arange(-180., 181., 20.))
    # m.drawmapboundary(fill_color='aqua')
    

    # make m a plot of the lat-lon map for one of the layers in antarctica centered around the crossover point
    # m should be an orthographic map centered around the crossover point
    # print("Plotting lat-lon map...")
    # print("--------------------")
    lat_0 = intersection_points[cross_index][0]
    lon_0 = intersection_points[cross_index][1]
    m = Basemap(projection='ortho', lat_0=lat_0, lon_0=lon_0, llcrnrx=-100000, llcrnry=-100000, urcrnrx=100000, urcrnry=100000, 
            resolution = 'c')

    
    m.drawcoastlines()
    m.fillcontinents(color='grey', lake_color='aqua')
    m.drawparallels(np.arange(-80., 81., 20.))
    m.drawmeridians(np.arange(-180., 181., 20.))
    m.drawmapboundary(fill_color='aqua')



    # plot the flight path
    m.plot(layers[0].lon, layers[0].lat, latlon=True, color='lightgreen', linewidth=1)
    # plot the section of the flight path in the plot above
    m.plot(layers[0].lon[intersection_indices[0][0] - offset:intersection_indices[0][0] + offset],
           layers[0].lat[intersection_indices[0][0] - offset:intersection_indices[0][0] + offset], latlon=True,
           color='red', linewidth=1)
    m.plot(layers[0].lon[intersection_indices[0][1] - offset:intersection_indices[0][1] + offset],
           layers[0].lat[intersection_indices[0][1] - offset:intersection_indices[0][1] + offset], latlon=True,
           color='green', linewidth=1)
    # plot labels for the flight paths at their start points
    plt.text(
        m(layers[0].lon[intersection_indices[0][0] - offset], layers[0].lat[intersection_indices[0][0] - offset])[
            0],
        m(layers[0].lon[intersection_indices[0][0] - offset], layers[0].lat[intersection_indices[0][0] - offset])[
            1], '\nsegment 1', fontsize='smaller', fontweight='bold', ha='right', va='top', color='red')
    plt.text(
        m(layers[0].lon[intersection_indices[0][1] - offset], layers[0].lat[intersection_indices[0][1] - offset])[
            0],
        m(layers[0].lon[intersection_indices[0][1] - offset], layers[0].lat[intersection_indices[0][1] - offset])[
            1], '\nsegment 2', fontsize='smaller', fontweight='bold', ha='left', va='top', color='green')
    # plot the South Pole
    m.scatter(0, -90, latlon=True, color='black', linewidth=1, label='South Pole')
    # plot the crossover points
    # for point in intersection_points:
    #     m.scatter(point[1], point[0], latlon=True, color='darkred', linewidth=1, label='Crossover Point')
    #     plt.text(m(point[1], point[0])[0], m(point[1], point[0])[1] - 10000, 'Crossover Point\n\n',
    #              fontsize='smaller', fontweight='bold', ha='center', va='top', color='darkred')
    
    m.scatter(intersection_points[cross_index][1], intersection_points[cross_index][0], latlon=True, color='darkred', linewidth=1, label='Crossover Point')
    plt.text(m(intersection_points[cross_index][1], intersection_points[cross_index][0])[0], m(intersection_points[cross_index][1], intersection_points[cross_index][0])[1] - 10000, 'Crossover Point\n\n',
                fontsize='smaller', fontweight='bold', ha='center', va='top', color='darkred')
    
    # plot the crossover line

    x, y = m(0, -90)
    plt.text(x, y, '\nSouth Pole', fontsize='smaller', fontweight='bold', ha='center', va='top', color='black')
    plt.title("Lat-Lon Map")
    # set tight layout
    plt.tight_layout()
    
    # plt.show()
    print("plotted map")
    print("--------------------\n")

    plt.show()
    
    # save the plot
    plt.savefig("layer_plot.png", dpi=1200)

In [5]:
print(season, flight)
cross_index = 0
plot_layers_at_cross(layers, intersection_indices, intersection_points,zoom=zoom, refractive_index=my_refractive_index,cross_index=cross_index)


# convert 25.8 microseconds to seconds
time = 25.8 * 10 ** -6
time2 = 30.5 * 10 ** -6

print(f"{twtt_to_depth(time, 1.77)} m")
print(f"{twtt_to_depth(time2, 1.77)} m")
print(f"difference: {twtt_to_depth(time, 1.77) - twtt_to_depth(time2, 1.77)} m")

2018_Antarctica_DC8 20181030_01


NameError: name 'my_refractive_index' is not defined

In [6]:
cross_index = 2
plot_layers_at_cross(layers, intersection_indices, intersection_points,zoom=zoom, refractive_index=my_refractive_index,cross_index=cross_index)

NameError: name 'my_refractive_index' is not defined

In [ ]:
# TODO: now that everything is in the library file -> look at the plot_layers_at_cross function and see if the corrected
    # layer portion is something that should be its own function and if it needs to be part of the posit object that is
    # being saved.

In [ ]:
# find the difference in lat-lon between each of the 4 points in the crossover
